In [12]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [13]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [14]:
# Get images
X = []
for filename in os.listdir('../Full-version/Train/'):
    X.append(img_to_array(load_img('../Full-version/Train/'+filename)))
    
X = np.array(X, dtype=float)

# Set up train and test data
Xtrain = X[:len(X) - 1]
Xtrain = 1.0/255*Xtrain

In [15]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [ ]:
# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=1000, steps_per_epoch=10)

Epoch 1/1000
10/10 [==============================] - 20s 1s/step - loss: 0.7068
Epoch 2/1000
10/10 [==============================] - 13s 1s/step - loss: 0.9988
Epoch 3/1000
10/10 [==============================] - 16s 2s/step - loss: 0.9974
Epoch 4/1000
10/10 [==============================] - 14s 1s/step - loss: 0.3756
Epoch 5/1000
10/10 [==============================] - 16s 2s/step - loss: 0.0128
Epoch 6/1000
10/10 [==============================] - 16s 2s/step - loss: 0.0122
Epoch 7/1000
10/10 [==============================] - 18s 2s/step - loss: 0.0384
Epoch 8/1000
10/10 [==============================] - 15s 1s/step - loss: 0.0130
Epoch 9/1000
10/10 [==============================] - 17s 2s/step - loss: 0.0124
Epoch 10/1000
10/10 [==============================] - 14s 1s/step - loss: 0.0136
Epoch 11/1000
10/10 [==============================] - 16s 2s/step - loss: 0.0118
Epoch 12/1000
10/10 [==============================] - 15s 1s/step - loss: 0.0105
Epoch 13/1000
10/10 [====

10/10 [==============================] - 15s 1s/step - loss: 0.0129
Epoch 102/1000
10/10 [==============================] - 14s 1s/step - loss: 0.0093
Epoch 103/1000
10/10 [==============================] - 15s 1s/step - loss: 0.0120
Epoch 104/1000
10/10 [==============================] - 13s 1s/step - loss: 0.0130
Epoch 105/1000
10/10 [==============================] - 16s 2s/step - loss: 0.0118
Epoch 106/1000
10/10 [==============================] - 14s 1s/step - loss: 0.0120
Epoch 107/1000
10/10 [==============================] - 15s 1s/step - loss: 0.0119
Epoch 108/1000
10/10 [==============================] - 13s 1s/step - loss: 0.0116
Epoch 109/1000
10/10 [==============================] - 14s 1s/step - loss: 0.0118
Epoch 110/1000
10/10 [==============================] - 13s 1s/step - loss: 0.0134
Epoch 111/1000
10/10 [==============================] - 15s 1s/step - loss: 0.0113
Epoch 112/1000
10/10 [==============================] - 13s 1s/step - loss: 0.0113
Epoch 113/1000
10/1

In [5]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [6]:
# Test images
Xtest = rgb2lab(1.0/255*X[len(X) - 1:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[len(X) - 1:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

1/1 [==============================] - 0s 185ms/step - loss: 0.0062
0.006219873204827309


In [8]:
color_me = []
for filename in os.listdir('../Full-version/Test/'):
    color_me.append(img_to_array(load_img('../Full-version/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
